In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [57]:
ds = pd.read_excel('/content/test_kv100_pvl_233.xlsx', header = 0).drop(axis = 1, labels = ['Unnamed: 0'])
ds

,Value,kv100,kv40,1,10,137,138,142,144,15,...,7,700,732,738,76,792,804,81,812,84
0,16.242092,14.554054,151.405818,48,0,0,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0,0,0.0,0.000000,0.000000
1,16.114329,14.499633,151.582072,48,0,0,0,0.0,0.000031,0.167658,...,0.000000,0.000000,0.0,0.0,0.000000,0,0,0.0,0.000000,0.002755
2,16.294710,6.006998,38.944687,0,0,0,0,0.0,0.000000,0.213289,...,0.000000,0.000000,0.0,0.0,0.000000,0,0,0.0,0.000000,0.396569
3,16.560651,5.981434,42.577751,0,0,0,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0,0,0.0,0.000000,0.402442
4,16.454986,5.501127,34.662824,0,0,0,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0,0,0.0,0.000000,0.247413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,9.970515,4.435105,19.891131,0,0,0,0,0.0,0.000000,0.000000,...,83.957926,0.015129,0.0,0.0,0.000000,0,0,0.0,0.086903,0.000000
339,7.557791,4.400000,20.500000,0,0,0,0,0.0,0.000000,88.519099,...,0.000000,0.000000,0.0,0.0,0.000000,0,0,0.0,0.000000,0.000000
340,8.815016,4.430919,19.909442,0,0,0,0,0.0,0.000000,0.445308,...,86.039878,0.020050,0.0,0.0,0.000704,0,0,0.0,0.000000,0.000000
341,10.284939,4.398986,23.058840,0,0,0,0,0.0,0.000000,49.222520,...,0.000000,0.000000,0.0,0.0,0.000000,0,0,0.0,0.000000,0.000000


In [62]:
X = ds.drop(axis = 1, labels = ['Value'])
y = ds['Value']

# 1. PCA

In [63]:
x = StandardScaler().fit_transform(X)
pca = PCA(n_components=10)
X_pca =pd.DataFrame(pca.fit_transform(X))

In [65]:
X_pca

,0,1,2,3,4,5,6,7,8,9
0,116.332570,-32.226694,38.409395,7.266940,-16.655399,4.856699,3.061451,-4.023571,1.003983,-0.495115
1,116.482763,-32.368557,38.520833,6.934394,-16.033777,4.468013,2.960903,-3.766415,1.229848,-0.529532
2,21.397393,9.586973,-30.446742,11.212896,-4.654128,3.812679,42.499279,36.711211,-17.383359,5.123686
3,24.450788,8.513980,-28.978198,11.571789,-5.087763,3.589085,41.457220,36.429157,-16.955401,5.475939
4,9.049816,15.961497,-37.935925,-35.385563,-30.885344,-13.330201,-24.657745,-1.189894,0.726286,-0.415587
...,...,...,...,...,...,...,...,...,...,...
338,-5.975885,81.063168,22.273732,3.860118,0.891392,-9.035921,10.093978,-9.322048,-3.704089,1.332897
339,-51.886450,-30.700495,19.552749,8.250294,-0.007626,-15.636426,3.597785,-1.422674,-3.031008,0.837540
340,-5.835616,82.217505,22.714886,3.937930,0.794530,-9.664633,10.576431,-10.198806,-4.262431,1.729735
341,-28.573930,-9.276607,-4.743191,-6.482021,-11.109909,-11.650371,5.930397,-3.733702,20.871551,-3.787721


In [66]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["Feature"] = X_pca.columns
vif_data["VIF"] = [variance_inflation_factor(X_pca.values, i)
                          for i in range(len(X_pca.columns))]
vif_data

,Feature,VIF
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0
5,5,1.0
6,6,1.0
7,7,1.0
8,8,1.0
9,9,1.0


In [67]:
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
vif_data

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


,Feature,VIF
0,kv100,301.404248
1,kv40,112.976467
2,1,35.351239
3,10,6.326954
4,137,3.156268
...,...,...
61,792,9.295732
62,804,1.454357
63,81,1430.912099
64,812,448.892221


# 2. Подбор алгоритма

In [9]:
pip install optuna

In [10]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import optuna

In [11]:
def objective(trial):
    rf_max_depth = trial.suggest_int('rf_max_depth', 3, 15)
    rf_min_samples_leaf = trial.suggest_float('rf_min_samples_leaf', 0.01, 1.0)
    rf_min_samples_split = trial.suggest_float('rf_min_samples_split', 0.01, 1.0)
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 100, 1000, step=100)

    gbr_n_estimators = trial.suggest_int('gbr_n_estimators', 100, 1000, step=100)
    gbr_max_depth = trial.suggest_int('gbr_max_depth', 3, 15)

    abr_n_estimators = trial.suggest_int('abr_n_estimators', 10, 960, step=50)

    knn_n_neighbors = trial.suggest_int('knn_n_neighbors', 2, 15)

    rf = RandomForestRegressor(max_depth=rf_max_depth, min_samples_leaf=rf_min_samples_leaf,
                               min_samples_split=rf_min_samples_split, n_estimators=rf_n_estimators,
                               n_jobs=-1, random_state=42)

    gbr = GradientBoostingRegressor(n_estimators=gbr_n_estimators, max_depth=gbr_max_depth, random_state=42)

    abr = AdaBoostRegressor(n_estimators=abr_n_estimators, random_state=42)

    knn = KNeighborsRegressor(n_neighbors=knn_n_neighbors)

    svr_c = trial.suggest_float('C', 1e-1, 10.0, log=False)
    svr_gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    svr_kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    svr_epsilon = trial.suggest_float('epsilon', 1e-3, 10.0, log=False)

    svr = SVR(C=svr_c, gamma=svr_gamma, kernel=svr_kernel, epsilon=svr_epsilon)


    ensemble_regressor = VotingRegressor(estimators=[
        ('rf', rf),
        ('gbr', gbr),
        ('abr', abr),
        ('svr', svr),
        ('knn', knn),
    ])

    pipeline = Pipeline([
        ('ensemble', ensemble_regressor)
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42) #PCA

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

[I 2024-03-06 12:55:59,522] A new study created in memory with name: no-name-ddaef6a1-1c1f-4190-8d3f-5a930cff736b
[I 2024-03-06 12:56:09,879] Trial 0 finished with value: 2.0377810637054594 and parameters: {'rf_max_depth': 13, 'rf_min_samples_leaf': 0.6538917133105978, 'rf_min_samples_split': 0.4146435767407535, 'rf_n_estimators': 500, 'gbr_n_estimators': 800, 'gbr_max_depth': 3, 'abr_n_estimators': 960, 'knn_n_neighbors': 7, 'C': 1.2635326956137076, 'gamma': 'auto', 'kernel': 'linear', 'epsilon': 6.934351294006711}. Best is trial 0 with value: 2.0377810637054594.
[I 2024-03-06 12:56:13,943] Trial 1 finished with value: 2.441459896628762 and parameters: {'rf_max_depth': 3, 'rf_min_samples_leaf': 0.4272085866724771, 'rf_min_samples_split': 0.037454418013212414, 'rf_n_estimators': 800, 'gbr_n_estimators': 200, 'gbr_max_depth': 12, 'abr_n_estimators': 310, 'knn_n_neighbors': 4, 'C': 8.827818607231602, 'gamma': 'scale', 'kernel': 'poly', 'epsilon': 8.016264881487329}. Best is trial 0 with 

In [14]:
best_params = study.best_params

best_rf_params = {
    'max_depth': best_params['rf_max_depth'],
    'min_samples_leaf': best_params['rf_min_samples_leaf'],
    'min_samples_split': best_params['rf_min_samples_split'],
    'n_estimators': best_params['rf_n_estimators'],
    'n_jobs': -1,
    'random_state': 42
}

best_gbr_params = {
    'n_estimators': best_params['gbr_n_estimators'],
    'max_depth': best_params['gbr_max_depth'],
    'random_state': 42
}

best_abr_params = {
    'n_estimators': best_params['abr_n_estimators'],
    'random_state': 42
}

best_knn_params = {
    'n_neighbors': best_params['knn_n_neighbors']
}

best_svr_params = {
        'C':best_params['C'],
        'gamma':best_params['gamma'],
        'kernel': best_params['kernel'],
        'epsilon': best_params['epsilon']
}
best_rf = RandomForestRegressor(**best_rf_params)
best_gbr = GradientBoostingRegressor(**best_gbr_params)
best_abr = AdaBoostRegressor(**best_abr_params)
best_knn = KNeighborsRegressor(**best_knn_params)
best_svr = SVR(**best_svr_params)

best_ensemble_regressor = VotingRegressor(estimators=[
    ('rf', best_rf),
    ('gbr', best_gbr),
    ('abr', best_abr),
    ('svr', best_svr),
    ('knn', best_knn)
])

In [15]:
models = [best_ensemble_regressor, best_rf, best_gbr, best_abr, best_knn, best_svr] #PCA
for model in models:
  model.fit(X_train, y_train)
  print(model.score(X_test, y_test))

0.7459805652867713
0.33726241683688873
0.8145022899341599
0.6925498381328395
0.6989963121804668
0.259868143481764


In [16]:
best_ensemble_regressor

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(max_depth=6,
                                                   min_samples_leaf=0.24051725854753564,
                                                   min_samples_split=0.3404390948446664,
                                                   n_estimators=500, n_jobs=-1,
                                                   random_state=42)),
                            ('gbr',
                             GradientBoostingRegressor(max_depth=5,
                                                       random_state=42)),
                            ('abr',
                             AdaBoostRegressor(n_estimators=660,
                                               random_state=42)),
                            ('svr',
                             SVR(C=2.3990818184832285,
                                 epsilon=3.9070507930185543, gamma='auto')),
                            ('knn', KNeighborsRegressor(n_neighbors=7))])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) #Standardized
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2024-03-06 12:57:32,074] A new study created in memory with name: no-name-36ea8a33-9e6f-4d78-9c94-62074fa50533
[I 2024-03-06 12:57:47,381] Trial 0 finished with value: 1.9131937443224456 and parameters: {'rf_max_depth': 10, 'rf_min_samples_leaf': 0.5801113567488382, 'rf_min_samples_split': 0.8364197094510619, 'rf_n_estimators': 900, 'gbr_n_estimators': 900, 'gbr_max_depth': 11, 'abr_n_estimators': 310, 'knn_n_neighbors': 4, 'C': 4.57389898144856, 'gamma': 'scale', 'kernel': 'rbf', 'epsilon': 3.996091827788703}. Best is trial 0 with value: 1.9131937443224456.
[I 2024-03-06 12:57:56,071] Trial 1 finished with value: 1.530310162954359 and parameters: {'rf_max_depth': 11, 'rf_min_samples_leaf': 0.5790236201296993, 'rf_min_samples_split': 0.9174630419409099, 'rf_n_estimators': 1000, 'gbr_n_estimators': 300, 'gbr_max_depth': 9, 'abr_n_estimators': 610, 'knn_n_neighbors': 4, 'C': 4.501781699340648, 'gamma': 'auto', 'kernel': 'linear', 'epsilon': 3.4389654354589387}. Best is trial 1 with va

In [18]:
best_params = study.best_params

best_rf_params = {
    'max_depth': best_params['rf_max_depth'],
    'min_samples_leaf': best_params['rf_min_samples_leaf'],
    'min_samples_split': best_params['rf_min_samples_split'],
    'n_estimators': best_params['rf_n_estimators'],
    'n_jobs': -1,
    'random_state': 42
}

best_gbr_params = {
    'n_estimators': best_params['gbr_n_estimators'],
    'max_depth': best_params['gbr_max_depth'],
    'random_state': 42
}

best_abr_params = {
    'n_estimators': best_params['abr_n_estimators'],
    'random_state': 42
}

best_knn_params = {
    'n_neighbors': best_params['knn_n_neighbors']
}

best_svr_params = {
        'C':best_params['C'],
        'gamma':best_params['gamma'],
        'kernel': best_params['kernel'],
        'epsilon': best_params['epsilon']
}
best_rf = RandomForestRegressor(**best_rf_params)
best_gbr = GradientBoostingRegressor(**best_gbr_params)
best_abr = AdaBoostRegressor(**best_abr_params)
best_knn = KNeighborsRegressor(**best_knn_params)
best_svr = SVR(**best_svr_params)

best_ensemble_regressor = VotingRegressor(estimators=[
    ('rf', best_rf),
    ('gbr', best_gbr),
    ('abr', best_abr),
    ('svr', best_svr),
    ('knn', best_knn)
])

In [19]:
models = [best_ensemble_regressor, best_rf, best_gbr, best_abr, best_knn, best_svr] #StandardScaler
for model in models:
  model.fit(X_train, y_train)
  print(model.score(X_test, y_test))

0.8399057745778558
-0.019834490440427066
0.8683114213531558
0.6781067875133558
0.6701513390731793
0.9358210149367596


In [20]:
best_ensemble_regressor

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(max_depth=11,
                                                   min_samples_leaf=0.3647444149197255,
                                                   min_samples_split=0.48339710518083695,
                                                   n_estimators=500, n_jobs=-1,
                                                   random_state=42)),
                            ('gbr',
                             GradientBoostingRegressor(max_depth=5,
                                                       n_estimators=600,
                                                       random_state=42)),
                            ('abr',
                             AdaBoostRegressor(n_estimators=610,
                                               random_state=42)),
                            ('svr',
                             SVR(C=2.7768946527638567,
                                 epsilon=0.8611317358834062, gamma='auto',
                                 kernel='linear')),
                            ('knn', KNeighborsRegressor(n_neighbors=2))])

In [71]:
pipe_SVR = Pipeline([('scaler', StandardScaler()), ('svr', SVR(**best_svr_params))]) # для модели без обработки PCA

pipe_GBR = Pipeline([('scaler', StandardScaler()), PCA(n_components = 10), ('svr', GradientBoostingRegressor(**best_gbr_params))]) # для модели c обработкой PCA


In [74]:
pipe_GBR.fit(X_train, y_train)
pipe_GBR.predict(X_test)

array([12.28359793, 11.46221543, 15.76921842, 13.20502858, 12.74472542,
       15.63207586, 14.56052249, 15.17624406, 10.45232443, 11.7736104 ,
       14.3045689 , 10.72467523, 14.51612669, 19.43831738, 13.92193318,
        7.93408237, 15.68150129, 15.43690407, 11.82591777, 12.25410307,
       10.41584723, 15.54579709, 15.02002371, 13.3788999 , 11.51224613,
       12.00993292, 11.5278066 , 11.51224613, 12.18766312, 11.34699196,
       14.29535487, 14.83992256, 13.65216674, 12.77083553, 12.69021259,
       14.13464478, 14.90787015, 20.60827537, 15.04252376, 14.93420094,
       18.76554279, 13.81949096,  9.98938064, 14.31755005, 13.65627034,
       14.84758467, 14.37738692, 10.41998181, 14.5922959 ,  9.05475059,
       10.08586119, 13.99148822, 13.85179338,  7.28565881, 13.22989029,
       14.38451621, 20.33978118, 10.86535403, 12.2847384 , 17.42309621,
       14.73608603, 11.37402373, 10.68516723, 14.19286996])